In [1]:
import os
import glob
import open3d
import numpy as np

from PIL import Image
from copy import deepcopy
from utils.depth_camera import DepthCamera
from utils.pointcloud import remove_outliers

In [3]:
def read_first_depth_img(experiment_dir, device_id, aligned):
    """
    Read the first depth image from a trial.
    """
    depth_img_path = glob.glob(f"{experiment_dir}/trial_0/global/device-{device_id}/*.depth.png")[0]
    device = DepthCamera(f"device-{device_id}", f"{experiment_dir}/metadata/device-{device_id}{'-aligned' if aligned else ''}.json")
    return device.depth_to_point_cloud(depth_img_path)


def read_extrinsics(experiment_dir, device_id):
    """
    Read the extrinsics from a trial.
    """
    return np.loadtxt(f"{experiment_dir}/trial_1/global/transformations/device-{device_id}.txt")


def write_extrinsics(experiment_dir, trial, device_id, transformation):
    """
    write the extrinsics from a trial.
    """
    if not os.path.exists(f"{experiment_dir}/{trial}/global/transformations"):
        os.makedirs(f"{experiment_dir}/{trial}/global/transformations")
        
    return np.savetxt(f"{experiment_dir}/{trial}/global/transformations/device-{device_id}.txt", transformation)


def execute_global_registration(source_down, target_down, source_feat, target_feat, n_ransac, threshold):
    result = open3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_feat, target_feat, threshold,
        open3d.registration.TransformationEstimationPointToPoint(False), n_ransac, 
        [open3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), open3d.registration.CorrespondenceCheckerBasedOnDistance(threshold)],
        open3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result


def refine_registration(source, target, distance_threshold, trans_init):
    result = open3d.registration.registration_icp(
        source, target, distance_threshold, trans_init,
        open3d.registration.TransformationEstimationPointToPlane(),
        open3d.registration.ICPConvergenceCriteria(max_iteration=1000)
    )
    return result


def visualize(source, target, transformation):
    source_temp = deepcopy(source)
    target_temp = deepcopy(target)
    
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    
    source_temp.transform(transformation)
    
    open3d.visualization.draw_geometries([source_temp, target_temp])
    
    
def rotate_transformation_matrix(t, rx, ry, rz):
    # Convert degrees to radians
    rx, ry, rz = np.radians(rx), np.radians(ry), np.radians(rz)

    RX = np.array([
        [1, 0, 0, 0],
        [0, np.cos(rx), -np.sin(rx), 0],
        [0, np.sin(rx), np.cos(rx), 0],
        [0, 0, 0, 1]
    ])

    RY = np.array([
        [np.cos(ry), 0, np.sin(ry), 0],
        [0, 1, 0, 0],
        [-np.sin(ry), 0, np.cos(ry), 0],
        [0, 0, 0, 1]
    ])

    RZ = np.array([
        [np.cos(rz), -np.sin(rz), 0, 0],
        [np.sin(rz), np.cos(rz), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    return np.dot(np.dot(np.dot(t, RZ), RY), RX)

In [18]:
dev_0_pcd = read_first_depth_img("data/raw_data/exp_12", 0, aligned=False)
dev_1_pcd = read_first_depth_img("data/raw_data/exp_12", 1, aligned=False)
dev_2_pcd = read_first_depth_img("data/raw_data/exp_12", 2, aligned=False)

In [6]:
open3d.io.write_point_cloud("temp/dev_0.pcd", dev_0_pcd)
open3d.io.write_point_cloud("temp/dev_1.pcd", dev_1_pcd)
open3d.io.write_point_cloud("temp/dev_2.pcd", dev_2_pcd)

True

In [12]:
dev_0_trans_init = np.loadtxt("temp/dev_0.txt")
dev_1_trans_init = np.loadtxt("temp/dev_1.txt")
dev_2_trans_init = np.loadtxt("temp/dev_2.txt")

In [13]:
dev_0_pcd = open3d.voxel_down_sample(dev_0_pcd, 0.03)
dev_1_pcd = open3d.voxel_down_sample(dev_1_pcd, 0.03)
dev_2_pcd = open3d.voxel_down_sample(dev_2_pcd, 0.03)

open3d.geometry.estimate_normals(dev_0_pcd)
open3d.geometry.estimate_normals(dev_1_pcd)
open3d.geometry.estimate_normals(dev_2_pcd)

True

In [7]:
dev_0_trans_init = read_extrinsics("data/raw_data/exp_11", 0)
dev_1_trans_init = read_extrinsics("data/raw_data/exp_11", 1)
dev_2_trans_init = read_extrinsics("data/raw_data/exp_11", 2)

In [19]:
dev_0_pcd.transform(dev_0_trans_init)
dev_1_pcd.transform(dev_1_trans_init)
dev_2_pcd.transform(dev_2_trans_init)

geometry::PointCloud with 109636 points.

In [15]:
open3d.visualization.draw_geometries([dev_0_pcd, dev_1_pcd, dev_2_pcd])

In [21]:
global_merged_pcd = open3d.geometry.PointCloud()

global_merged_pcd += dev_0_pcd
global_merged_pcd += dev_1_pcd
global_merged_pcd += dev_2_pcd

global_merged_pcd = open3d.voxel_down_sample(global_merged_pcd, 0.015)

global_merged_pcd = remove_outliers(global_merged_pcd)

open3d.visualization.draw_geometries([global_merged_pcd])

open3d.io.write_point_cloud("data/reference/larc_kitchen_v5.pcd", global_merged_pcd)

# open3d.io.write_point_cloud("data/reference/larc_kitchen_3cams.pcd", global_merged_pcd)

True

In [22]:
for trial in os.listdir("data/raw_data/exp_12/"):
    if trial.startswith("trial"):
        write_extrinsics("data/raw_data/exp_12", trial, 0, dev_0_trans_init)
        write_extrinsics("data/raw_data/exp_12", trial, 1, dev_1_trans_init)
        write_extrinsics("data/raw_data/exp_12", trial, 2, dev_2_trans_init)

In [1]:
import open3d
import numpy as np
import pandas as pd
import os
import tqdm
import copy
import glob

from scipy.signal import argrelmin
from PIL import Image

import utils.registration as registration
import utils.grid_search_unopt as grid_search
import utils.fread as fread
import utils.FCGF as FCGF
import utils.registration as registration
import utils.functions as functions
import utils.transform as transform
import utils.pointcloud as pointcloud

In [2]:
def find_candidate_global_pos(std_values, delta):
    global_pos = [0]
    prev_t = 0
    for current_t in range(len(std_values)):
        if np.abs(std_values[current_t] - std_values[prev_t]) > delta:
            global_pos.append(current_t)
            prev_t = current_t
            
    return global_pos


def find_cutoffs(std_values, target_fps, min_std, threshold):
    cutoffs = argrelmin(std_values, order=target_fps // 2)[0]
    return cutoffs[np.where(np.abs(std_values[cutoffs] - min_std) < threshold)[0]]

In [3]:
min_std = 0.5
threshold = 0.5
target_fps = 20
cutoff_margin = 5 # frames

experiment = "exp_12"
subject = "subject-1"
trial = "trial_1"
sequence = "01"

voxel_size = 0.03

out_dir = "data/trajectories/groundtruth/exp_12"

sequence_dir = f"data/raw_data/{experiment}/{trial}/secondary/{subject}/{sequence}/frames"
feature_dir = os.path.join("data/features", experiment, trial, str(voxel_size), subject, sequence)

sequence_ts = fread.get_timstamps(feature_dir, ext=".secondary.npz")
num_frames = len(sequence_ts)

file_name = f"{experiment}__{trial}__{subject}__{sequence}"

In [4]:
print(num_frames)

651


In [5]:
std_values = []

for t in range(len(sequence_ts)):
    depth_img = Image.open(os.path.join(sequence_dir, f"frame-{sequence_ts[t]}.depth.png")).convert("I")
    depth_img = np.array(depth_img) / 4000
    std_values.append(np.std(depth_img))
    
std_values = np.array(std_values)

cutoffs = find_cutoffs(std_values, target_fps, min_std, threshold)
cutoffs = np.concatenate([[0], cutoffs, [num_frames - 1]])
cutoffs = list(zip(cutoffs[:-1] + cutoff_margin, cutoffs[1:] - cutoff_margin))

In [8]:
local_t = np.load(os.path.join(f"data/trajectories/groundtruth/{experiment}", f"{file_name}.pose.npz"))["local_t"]

fragment_files = glob.glob(os.path.join(f"data/fragments/{experiment}", f"{file_name}__*.pcd"))
fragment_files = sorted(fragment_files, key=lambda f: int(os.path.basename(f).split(".")[0].split("__")[-1]))

fragment_t = [np.loadtxt(fragment_files[i].replace("pcd", "txt")) for i in range(len(fragment_files))]

trajectory_t = [np.identity(4) for _ in range(num_frames)]
global_t = [np.zeros((4, 4)) for _ in range(num_frames)]

In [9]:
fragment_files

['data/fragments/exp_12\\exp_12__trial_1__subject-1__01__00.pcd',
 'data/fragments/exp_12\\exp_12__trial_1__subject-1__01__01.pcd',
 'data/fragments/exp_12\\exp_12__trial_1__subject-1__01__02.pcd',
 'data/fragments/exp_12\\exp_12__trial_1__subject-1__01__03.pcd',
 'data/fragments/exp_12\\exp_12__trial_1__subject-1__01__04.pcd']

In [10]:
if len(fragment_t) < len(cutoffs):
    fragment_ids = [int(os.path.basename(f).split(".")[0].split("__")[-1]) for f in fragment_files]
    cutoffs = [cutoffs[fi] for fi in fragment_ids]

for fragment_ind, (start_t, end_t) in enumerate(cutoffs):
    for t in range(start_t + 1, end_t):
        trajectory_t[t] = np.dot(trajectory_t[t - 1], local_t[t])
        
    for t in range(start_t, end_t):
        global_t[t] = np.dot(fragment_t[fragment_ind], trajectory_t[t])